In [12]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
from ir_measures import *
import pandas as pd

In [2]:
pt.init()
ensure_pyterrier_is_loaded()
tira = Client()


terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /home/codespace/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /home/codespace/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [10]:
pt_dataset = pt.get_dataset("irds:ir-lab-padua-2024/longeval-2023-01-20240426-training")
pt_dataset_jun = pt.get_dataset("irds:ir-lab-padua-2024/longeval-2023-06-20240418-training")
pt_dataset_aug = pt.get_dataset("irds:ir-lab-padua-2024/longeval-2023-08-20240418-training")

In [6]:
bm25 = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-01-20240423-training')
sce = tira.pt.from_submission('ir-benchmarks/fschlatt/sparse-cross-encoder-4-512', 'ir-benchmarks/longeval-2023-01-20240423-training')
cb = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/ColBERT Re-Rank (tira-ir-starter-pyterrier)', 'ir-benchmarks/longeval-2023-01-20240423-training')
rz = tira.pt.from_submission('workshop-on-open-web-search/fschlatt/rank-zephyr', 'ir-benchmarks/longeval-2023-01-20240423-training')
fusion = pt.transformer.get_transformer(pt.io.read_results('galapagos-tortoise-wsum.lag1'))

In [7]:
pt.Experiment(
    [
        bm25,
        sce,
        cb,
        rz,
        fusion,
    ],
    pt_dataset.get_topics('text'),
    pt_dataset.get_qrels(),
    names=[
        'BM25', 
        "Sparse Cross-Encoder",
        'ColBERT',
        'RankZephyr', 
        'WSum',
        # "3-max-avg",
    ],
    eval_metrics=[nDCG(judged_only=True)@10, nDCG(judged_only=True), nDCG@10, nDCG],
).sort_values(by="nDCG@10", ascending=False)

,name,nDCG(judged_only=True)@10,nDCG(judged_only=True),nDCG@10,nDCG
4,WSum,0.454667,0.480725,0.251512,0.355077
3,RankZephyr,0.455142,0.482076,0.246549,0.352920
1,Sparse Cross-Encoder,0.448109,0.474784,0.220869,0.336719
2,ColBERT,0.450091,0.476047,0.216457,0.330096
0,BM25,0.444411,0.472638,0.199380,0.317285


### Hypothesis Test on June-Dataset.

In [13]:
pt.Experiment(
    [
        bm25,
        sce,
        cb,
        rz,
        fusion,
    ],
    pt_dataset_jun.get_topics('text'),
    pt_dataset_jun.get_qrels(),
    #pd.read_csv("01_qrels_jun.txt"),
    names=[
        'BM25', 
        "Sparse Cross-Encoder",
        'ColBERT',
        'RankZephyr', 
        'WSum',
    ],
    eval_metrics=[nDCG(judged_only=True)@10, nDCG(judged_only=True), nDCG@10, nDCG],
    baseline = 4
).sort_values(by="nDCG", ascending=False)


The download is derived from The LongEval Dataset under the "Qwant LongEval Attribution-NonCommercial-ShareAlike License". Hence, the download is also under this License. By using it, you agree to the terms of this license. Please find details at: https://lindat.mff.cuni.cz/repository/xmlui/page/Qwant_LongEval_BY-NC-SA_License
Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-padua2024/longeval-2023-06-20240418-truth.zip?download=1
	This is only used for last spot checks before archival to Zenodo.
Code: 404
Error occured while fetching https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-padua2024/longeval-2023-06-20240418-truth.zip?download=1. I will sleep 14 seconds and continue.
Code: 404
Error occured while fetching https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-padua2024/longeval-2023-06-20240418-truth.zip?download=1. I will sle

FileNotFoundError: [Errno 2] No such file or directory: '/home/codespace/.tira/extracted_datasets/ir-lab-padua-2024/longeval-2023-06-20240418-training//longeval-2023-06-20240418-training' -> '/home/codespace/.tira/extracted_datasets/ir-lab-padua-2024/longeval-2023-06-20240418-training/truth-data'

### Hypothesis Test on August-Dataset.

In [ ]:
hypo_aug = pt.Experiment(
    [
        bm25,
        sce,
        cb,
        rz,
        fusion,
    ],
    pt_dataset_aug.get_topics('text'),
    pt_dataset_aug.get_qrels(),
    names=[
        'BM25', 
        "Sparse Cross-Encoder",
        'ColBERT',
        'RankZephyr', 
        'WSum',
    ],
    eval_metrics=[nDCG(judged_only=True)@10, nDCG(judged_only=True), nDCG@10, nDCG],
    baseline = 4
).sort_values(by="nDCG", ascending=False)

hypo_aug